# P307 KNN Vowel2
+ Try to improve F1 for Vowel2

In [6]:
import numpy as np
import pandas as pd
import helper
import submission
import re
import pickle

In [7]:
def split_to_list(curr_row):
    word = curr_row[:curr_row.index(':')]
    splitted_list = curr_row[curr_row.index(':') + 1 : ].split(' ')
    return word, splitted_list

def check(splitted_list):
    count = 0
    target = 0
    removed_stress = []
    for item in splitted_list[:]:
        try: # vowel
            curr = int(item[-1]) 
            removed_stress.append(item[:-1])
            count += 1
            if curr == 1:
                target = count
        except: # consonant
            removed_stress.append(item)
    return removed_stress, target
def count_vowel(removed_stress, vowel):
    count = 0
    for item in removed_stress[:]:
        if item in vowel:
            count += 1
    return count

In [11]:
def singular_filter(word, removed_stress):
    
    ## 这里修改下, 若单词长度 <= 3, 不进行消除
    if len(word) <= 3:
        return word, removed_stress
    
    sub_string1 = word[-2: ]
    sub_string2 = word[-3: ]
    sub_string3 = word[-6: ]    
    sub_string4 = word[-4: ]
    
    pattern1 = re.compile('[BCDFGHJKLMNOPQRTVWYZ]{1}S$')
    pattern2 = re.compile('XS$')
    # For CDFGHKLPWRTWYZ --> remove 'S' from word, remove 'S/Z' from removed_stress
    if re.match(pattern1, sub_string1):
        return word[:-1], removed_stress[:-1]
    # For X --> remove 'S' from word, remove 'IH Z' from removed_stress
    if re.match(pattern2, sub_string1):
        return word[:-1], removed_stress[:-2]
    
    # -ES
    pattern3 = re.compile('[ABDEFJKLMNOPQRTUWY]{1}ES$')
    pattern4 = re.compile('IES$')
    pattern5 = re.compile('HES$')
    pattern55 = re.compile('ZZES$|SSES$')
    pattern6 = re.compile('[CGSXZ]{1}ES$')
    pattern7 = re.compile('SELVES$')
    pattern8 = re.compile('IVES$')
    pattern9 = re.compile('VES$')
    
    if re.match(pattern3, sub_string2): # ES --> E
        return word[:-1], removed_stress[:-1]
    if re.match(pattern4, sub_string2): # IES --> Y
        word = word[:-3] + 'Y'
        return word, removed_stress[:-1]
    if re.match(pattern5, sub_string2):
        return word[:-2], removed_stress[:-2]
    if re.match(pattern55, sub_string4):
        return word[:-2], removed_stress[:-2]
    if re.match(pattern6, sub_string2):
        return word[:-1], removed_stress[:-2]
    if re.match(pattern7, sub_string3):
        word = word[:-6] + 'SELF'
        removed_stress.pop()
        removed_stress.pop()
        removed_stress.append('F')
        return word, removed_stress
    if re.match(pattern8, sub_string4) and (removed_stress[-3:] == ['AY', 'V', 'Z']):
        word = word[:-3] + 'FE'
        removed_stress.pop()
        removed_stress.pop()
        removed_stress.append('F')
        return word, removed_stress
    if re.match(pattern9, sub_string2):
        return word[:-1], removed_stress[:-1]
    
    return word, removed_stress

In [12]:
def neutral_filter1(word, removed_stress):
    pattern1 = re.compile('ABL[EY]{1}$')
    if re.search(pattern1, word):
        return word[:-4], removed_stress[:-4]
    
    pattern2 = re.compile('LL[YI]{1}$|LLED$|SSED$|FFED$')
    if re.search(pattern2, word):
        return word[:-2], removed_stress[:-1]
    
    pattern3 = re.compile('L[YI]{1}$')
    if re.search(pattern3, word) and removed_stress[-2:] == ['L', 'IY']:
        return word[:-2], removed_stress[:-2]
    
    pattern4 = re.compile('DDED$|TTED$')
    if re.search(pattern4, word):
        return word[:-3], removed_stress[:-2]
    
    pattern5 = re.compile('[AEIOU]{1}[DFT]{1}ED$')
    if re.search(pattern5, word):
        return word[:-1], removed_stress[:-2]
    
    pattern6 = re.compile('[AEIOU]{1}[KMNS]{1}ED$|[IU]{1}RED$|IBED$')
    if re.search(pattern6, word):
        return word[:-1], removed_stress[:-1]
    
    pattern7 = re.compile('BBED$|GGED$|MMED$|NNED$|PPED$|RRED$')
    if re.search(pattern7, word):
        return word[:-3], removed_stress[:-1]
    
    pattern8 = re.compile('[SW]{1}EED$')
    if re.search(pattern8, word):
        return word[:-4], removed_stress[:-3]
    
    pattern9 = re.compile('B[LR]{1}EED$')
    if re.search(pattern9, word):
        return word[:-5], removed_stress[:-4]
    
    pattern10 = re.compile('[BHKMNOPSWXY]{1}ED$')
    if re.search(pattern10, word):
        return word[:-2], removed_stress[:-1]
    
    pattern11 = re.compile('IED$')
    if re.search(pattern11, word):
        word = word[:-3] + 'Y'
        return word, removed_stress[:-1]
    
    pattern12 = re.compile('[CGLUVZ]{1}ED$')
    if re.search(pattern12, word):
        return word[:-1], removed_stress[:-1]
    
    pattern13 = re.compile('[DFTR]{1}ED$')
    if re.search(pattern13, word):
        return word[:-2], removed_stress[:-2]
    
    pattern14 = re.compile('ISM$')
    if re.search(pattern14, word):
        return word[:-3], removed_stress[:-4]    
    
    pattern15 = re.compile('FUL$|[AO]{1}LF$')
    if re.search(pattern15, word):
        return word[:-3], removed_stress[:-3]  
    
    pattern16 = re.compile('[NL]{1}ESS$') # remove 'NESS', remove 'N AH0 S'
    if re.search(pattern16, word):
        return word[:-4], removed_stress[:-3]
    
    pattern17 = re.compile('ING$')
    if re.search(pattern17, word):
        return word[:-3], removed_stress[:-2]
    
    return word, removed_stress

In [13]:
# 只有singular 和neutral 1
def combined_filter(word, removed_stress, vowel):
    
    if count_vowel(removed_stress, vowel) <= 2:
        return word, removed_stress
    word, removed_stress = singular_filter(word, removed_stress)
    if len(word) == 0 or len(removed_stress) == 0: 
        return 1, [1]
    
    if count_vowel(removed_stress, vowel) <= 2:
        return word, removed_stress
    word, removed_stress = neutral_filter1(word, removed_stress)
    if len(word) == 0 or len(removed_stress) == 0: 
        return 1, [1]
    
    count = 0
    for item in removed_stress[:]:
        if item in vowel:
            count += 1
    if count == 0:
        return 1, [1]
    
    return word, removed_stress

## 获取特征

In [14]:
def get_structure_and_vowels(removed_stress, vowel):
    structure = ''
    num_vowels = 0
    count = 0
    vowels = []
    vowel_positions = []
    for item in removed_stress[:]:
        if item in vowel:
            structure += 'V'
            vowels.append(vowel.get(item))
            vowel_positions.append(count)
            num_vowels += 1
            count += 1
        else:
            structure += 'C'
            count += 1
        
    return vowels, vowel_positions, num_vowels, structure

In [15]:
def get_prefix_and_suffix(curr_vowel_position, removed_stress, consonant):
    
    if curr_vowel_position == 0: # no prefixes
        prefix1 = 39
    else:
        prefix1 = consonant.get(removed_stress[curr_vowel_position - 1])
    
    try:
        suffix1 = consonant.get(removed_stress[curr_vowel_position + 1])
    except:
        suffix1 = 39
    return [prefix1, suffix1]


In [16]:
##这里加入新函数
def split_structure_vowel2(vowel_positions, removed_stress):
    split_result = []
    if vowel_positions[1] - vowel_positions[0] > 1: 
        split_result.append(' '.join(removed_stress[:vowel_positions[1] - 1]))
        split_result.append(' '.join(removed_stress[vowel_positions[1] - 1 : ]))
    else:
        split_result.append(' '.join(removed_stress[:vowel_positions[0] + 1]))
        split_result.append(' '.join(removed_stress[vowel_positions[0] + 1 : ]))
    return split_result
def split_structure_vowel3(vowel_positions, removed_stress):
    if vowel_positions[1] - vowel_positions[0] > 1: # CVCCCVXXXX
        part1 = removed_stress[:vowel_positions[1] - 1]
        removed_stress = removed_stress[vowel_positions[1] - 1 : ]
        
    else: # CVVXXXX
        part1 = removed_stress[:vowel_positions[0] + 1]
        removed_stress = removed_stress[vowel_positions[0] + 1 : ]
    
    vowel_positions = [vowel_positions[1] - len(part1), vowel_positions[2] - len(part1)]
    sub_split_result = split_structure_vowel2(vowel_positions, removed_stress)
    split_result = [' '.join(part1)]
    split_result.extend(sub_split_result)
    return split_result
def split_structure_vowel4(vowel_positions, removed_stress):
    if vowel_positions[1] - vowel_positions[0] > 1: # CVCCCVXXXX
        part1 = removed_stress[:vowel_positions[1] - 1]
        removed_stress = removed_stress[vowel_positions[1] - 1 : ]
        
    else: # CVVXXXX
        part1 = removed_stress[:vowel_positions[0] + 1]
        removed_stress = removed_stress[vowel_positions[0] + 1 : ]
    
    vowel_positions = [vowel_positions[1] - len(part1), vowel_positions[2] - len(part1), vowel_positions[3] - len(part1)]
    sub_split_result = split_structure_vowel3(vowel_positions, removed_stress)
    split_result = [' '.join(part1)]
    split_result.extend(sub_split_result)
    return split_result

## 将数据先按照元音数量进行切分, 然后按训练集测试集进行切分

In [17]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score,precision_score, recall_score
vowel = {'AA': 0, 'AW': 1, 'AY': 2, 'ER': 3, 'EY': 4, 'IY': 5, 'OW': 6, 'OY': 7, 'UW': 8, 'AE': 9, 'AH': 10, 'AO': 11, 'EH': 12, 'IH': 13, 'UH': 14}
consonant = {'AA': 0, 'AW': 1, 'AY': 2, 'ER': 3, 'EY': 4, 'IY': 5, 'OW': 6, 'OY': 7, 'UW': 8, 'AE': 9, 'AH': 10, 'AO': 11, 'EH': 12, 'IH': 13, 'UH': 14, 'P': 15, 'B': 16, 'CH': 17, 'D': 18, 'DH': 19, 'F': 20, 'G': 21, 'HH': 22, 'JH': 23, 'K': 24, 'L': 25, 'M': 26, 'N': 27, 'NG': 28, 'R': 29, 'S': 30, 'SH': 31, 'T': 32, 'TH': 33, 'V': 34, 'W': 35, 'Y': 36, 'Z': 37, 'ZH': 38}

In [18]:
raw_data = helper.read_data('asset/training_data.txt')

In [19]:
# split raw data by vowel
def split_raw_by_vowels(raw_data, vowel):
    vowel2 = []
    vowel3 = []
    vowel4 = []
    for curr_row in raw_data[:]:
        word, splitted_list = split_to_list(curr_row)
        removed_stress, target = check(splitted_list)
        num_vowel = count_vowel(removed_stress, vowel)
        if num_vowel == 2:
            vowel2.append(word + ':' + ' '.join(splitted_list))
        if num_vowel == 3:
            vowel3.append(word + ':' + ' '.join(splitted_list))
        if num_vowel == 4:
            vowel4.append(word + ':' + ' '.join(splitted_list))
    return vowel2, vowel3, vowel4

In [20]:
vowel2, vowel3, vowel4 = split_raw_by_vowels(raw_data, vowel)
print(len(vowel2))
print(len(vowel3))
print(len(vowel4))

27619
16395
5986


In [21]:
def get_structure_and_vowels2(removed_stress, vowel, consonant):
    structure = ''
    num_vowels = 0
    count = 0
    vowels = []
    vowel_positions = []
    VowelMap = []
    ConsonantMap = []
    VectorMap = [0] * 39
    VectorMapCount = [0] * 39
    for item in removed_stress[:]:
        vowel_index = consonant.get(item)
        VectorMap[vowel_index] = 1
        VectorMapCount[vowel_index] += 1
        if item in vowel:
            structure += 'V'
            vowels.append(vowel_index)
            VowelMap.append(1)
            ConsonantMap.append(0)
            vowel_positions.append(count)
            num_vowels += 1
            count += 1
        else:
            VowelMap.append(0)
            ConsonantMap.append(1)
            structure += 'C'
            count += 1
            
    if count < 15:
        VowelMap += [0] * (15 - count)
        ConsonantMap += [0] * (15 - count)
            
    return vowels, vowel_positions, structure, VowelMap, ConsonantMap, VectorMap, VectorMapCount

In [22]:
# Test
rss = ['N', 'AA', 'N', 'P', 'OY', 'Z', 'AH', 'N', 'AH', 'S']
vowels, vowel_positions, structure, VowelMap, ConsonantMap, VectorMap, VectorMapCount = get_structure_and_vowels2(rss, vowel, consonant)
for item in [vowels, vowel_positions, structure, VowelMap, ConsonantMap, VectorMap, VectorMapCount]:
    print(item)

[0, 7, 10, 10]
[1, 4, 6, 8]
CVCCVCVCVC
[0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]


In [107]:
# Test
split_structure_vowel4(vowel_positions, rss)

['N AA N', 'P OY', 'Z AH', 'N AH S']

In [23]:
def get_features_vowel2(vowel2, vowel, consonant):
    data2 = []
    structure_dict = {}
    syllable_dict = {}
    
    for curr_row in vowel2[:]:
        word, splitted_list = split_to_list(curr_row)
        removed_stress, target = check(splitted_list)
        word, removed_stress = combined_filter(word, removed_stress, vowel)
        # word, removed_stress = singular_filter(word, removed_stress)
        
        if type(word) == int:
            continue
        else:
            vowels, vowel_positions, structure, VowelMap, ConsonantMap, VectorMap, VectorMapCount = get_structure_and_vowels2(removed_stress, vowel, consonant)
            
            # some word may be removed after filtering
            if len(vowels) != 2:
                continue
            
            # get the prefix and suffix
            prefix_suffix = []
            for curr_vowel_position in vowel_positions[:]:
                p_s = get_prefix_and_suffix(curr_vowel_position, removed_stress, consonant)
                prefix_suffix.extend(p_s)
            
            # get the structure of vowel
            
            if structure in structure_dict:
                structure_index = structure_dict.get(structure)
            else:
                structure_index = len(structure_dict)
                structure_dict[structure] = structure_index
            
            # get the syllables
            split_result = split_structure_vowel2(vowel_positions, removed_stress)
            syllable_indexes = []
            for syllable in split_result[:]:
                if syllable in syllable_dict:
                    syllable_index = syllable_dict.get(syllable)
                else:
                    syllable_index = len(syllable_dict)
                    syllable_dict[syllable] = syllable_index
                syllable_indexes.append(syllable_index)
            
            # build the row data
            row_data = [target, word, ' '.join(removed_stress), structure, structure_index, vowels, prefix_suffix, VowelMap, ConsonantMap, VectorMap, split_result, syllable_indexes]
            
            data2.append(row_data)
    
    return data2, structure_dict, syllable_dict

In [24]:
data2, structure_dict, syllable_dict = get_features_vowel2(vowel2, vowel, consonant)

In [25]:
len(data2)

27619

In [28]:
def get_df2(data2):
    features = ['Target','Word','RemovedStress', 'Structure', 'StructureIndex', 'Vowels', 'PrefixSuffix', 'VowelPositionMap', 'ConsonantPositionMap', 'VectorMap', 'Syllables','SyllableIndex']
    df = pd.DataFrame(data2, columns = features)
    return df

In [29]:
df2 = get_df2(data2)

In [31]:
df2[:3]

,Target,Word,RemovedStress,Structure,StructureIndex,Vowels,PrefixSuffix,VowelPositionMap,ConsonantPositionMap,VectorMap,Syllables,SyllableIndex
0,1,COED,K OW EH D,CVVC,0,"[6, 12]","[24, 12, 6, 18]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","[K OW, EH D]","[0, 1]"
1,1,PURVIEW,P ER V Y UW,CVCCV,1,"[3, 8]","[15, 34, 36, 39]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[P ER V, Y UW]","[2, 3]"
2,1,HEHIR,HH EH HH IH R,CVCVC,2,"[12, 13]","[22, 22, 22, 29]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...","[HH EH, HH IH R]","[4, 5]"


## 得到训练模式

In [32]:
def get_combination(Syllables):
    combination = []
    length = len(Syllables)
    for curr_length in range(1,length + 1):
        for start_position in range(length - curr_length + 1):
            combination.insert(0, ' '.join(Syllables[start_position : start_position + curr_length]))
    return combination

In [33]:
# Test
Syls = ['N AY', 'JH IH', 'R IY', 'AH N']
get_combination(Syls)

['N AY JH IH R IY AH N',
 'JH IH R IY AH N',
 'N AY JH IH R IY',
 'R IY AH N',
 'JH IH R IY',
 'N AY JH IH',
 'AH N',
 'R IY',
 'JH IH',
 'N AY']

In [34]:
def vowels_to_bin(Vowels, vowel):
    result = []
    for item in Vowels[:]:
        curr = [0] * 15
        curr[item] += 1
        result.extend(curr)
    return result

In [35]:
def prefix_suffix_to_bin(PrefixSuffix, consonant):
    result = []
    for item in PrefixSuffix[:]:
        curr = [0] * 40 # note "39" is for boundary
        curr[item] += 1
        result.extend(curr)
    return result

In [36]:
df2['VowelMap'] = df2.Vowels.apply(vowels_to_bin, vowel = vowel)
df2['PrefixSuffixMap'] = df2.PrefixSuffix.apply(prefix_suffix_to_bin, consonant = consonant)
df2['SyllableCombination'] = df2.Syllables.apply(get_combination)

In [37]:
df2[:3]

,Target,Word,RemovedStress,Structure,StructureIndex,Vowels,PrefixSuffix,VowelPositionMap,ConsonantPositionMap,VectorMap,Syllables,SyllableIndex,VowelMap,PrefixSuffixMap,SyllableCombination
0,1,COED,K OW EH D,CVVC,0,"[6, 12]","[24, 12, 6, 18]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","[K OW, EH D]","[0, 1]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[K OW EH D, EH D, K OW]"
1,1,PURVIEW,P ER V Y UW,CVCCV,1,"[3, 8]","[15, 34, 36, 39]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[P ER V, Y UW]","[2, 3]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[P ER V Y UW, Y UW, P ER V]"
2,1,HEHIR,HH EH HH IH R,CVCVC,2,"[12, 13]","[22, 22, 22, 29]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...","[HH EH, HH IH R]","[4, 5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[HH EH HH IH R, HH IH R, HH EH]"


In [41]:
len(df2.PrefixSuffixMap[4])

160

## Unpacked Dataset for training

In [43]:
unpacked_dataset0 = pd.DataFrame.from_records(df2.SyllableCombination.tolist(),columns = ['C12','C2','C1'])
unpacked_dataset1 = pd.DataFrame.from_records(df2.VowelPositionMap.tolist(),columns = range(1,16))
unpacked_dataset2 = pd.DataFrame.from_records(df2.ConsonantPositionMap.tolist(),columns = range(16,31))
unpacked_dataset3 = pd.DataFrame.from_records(df2.VectorMap.tolist(),columns = range(31, 70))
unpacked_dataset4 = pd.DataFrame.from_records(df2.VowelMap.tolist(),columns = range(100,130))
unpacked_dataset5 = pd.DataFrame.from_records(df2.PrefixSuffixMap.tolist(),columns = range(1000,1160))

In [44]:
final_dataset = pd.concat([df2.Target, unpacked_dataset0], axis=1) # Syllable Combinations
final_dataset = pd.concat([final_dataset, unpacked_dataset1], axis=1) # VowelPositionMap
final_dataset = pd.concat([final_dataset, unpacked_dataset2], axis=1) # ConsonantPositionMap
final_dataset = pd.concat([final_dataset, unpacked_dataset3], axis=1) # VectorMap
final_dataset = pd.concat([final_dataset, unpacked_dataset4], axis=1) # VowelMap
final_dataset = pd.concat([final_dataset, unpacked_dataset5], axis=1) # PrefixSuffixMap

In [45]:
final_dataset[:3]

,Target,C12,C2,C1,1,2,3,4,5,6,...,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159
0,1,K OW EH D,EH D,K OW,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,P ER V Y UW,Y UW,P ER V,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,1,HH EH HH IH R,HH IH R,HH EH,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Make Prediction

In [60]:
train, test = train_test_split(final_dataset, test_size = 0.1)
features = list(final_dataset.columns)[4:]
train_X = train[features]
train_y = train['Target']
test_X = test[features]
test_y = test['Target']

In [61]:
for i in range(5, 100, 5):
    print ('k = ' + str(i))
    neigh = KNeighborsClassifier(n_neighbors=i, weights='distance')
    neigh.fit(train_X, train_y)
    print('Training acc : ' + str(neigh.score(train_X, train_y)))
    print('Testing acc : ' + str(neigh.score(test_X, test_y)))
    pred = neigh.predict(test_X)
    print('F1 : ' + str(f1_score(list(test_y), pred, average='macro')))

k = 5
Training acc : 0.998390795349
Testing acc : 0.929036929761


KeyboardInterrupt: 

In [57]:
## k = 5
f1 = 0
for round in range(1,6):
    print("Round : " + str(round))
    train, test = train_test_split(final_dataset, test_size = 0.1)
    features = list(final_dataset.columns)[4:]
    train_X = train[features]
    train_y = train['Target']
    test_X = test[features]
    test_y = test['Target']
    neigh = KNeighborsClassifier(n_neighbors=5, weights='distance')
    neigh.fit(train_X, train_y)
    print('Training acc : ' + str(neigh.score(train_X, train_y)))
    print('Testing acc : ' + str(neigh.score(test_X, test_y)))
    pred = neigh.predict(test_X)
    curr_f1 = f1_score(list(test_y), pred, average='macro')
    f1 += curr_f1
    print('F1 : ' + str(curr_f1))
    print("= = = = = Round End = = = = =")
print("Average F1 : " + str(f1 * 1.0 / 5))

Round : 1
Training acc : 0.998390795349
Testing acc : 0.933381607531
F1 : 0.802648399699
= = = = = Round End = = = = =
Round : 2
Training acc : 0.998390795349
Testing acc : 0.935916002896
F1 : 0.796155987116
= = = = = Round End = = = = =
Round : 3
Training acc : 0.998471255582
Testing acc : 0.92867487328
F1 : 0.799482966072
= = = = = Round End = = = = =
Round : 4
Training acc : 0.998431025466
Testing acc : 0.935191889935
F1 : 0.81097547221
= = = = = Round End = = = = =
Round : 5
Training acc : 0.998270105001
Testing acc : 0.929036929761
F1 : 0.804638567348
= = = = = Round End = = = = =
Average F1 : 0.802780278489


+ We can find that k = 7 is good, now we try to get more detailed dataset
+ 每次通过Syllables看能不能找到更精确的Syllables
## 对每个训练样本, 执行搜索

In [55]:
def get_refined_df(test_combination, df):
    df_features = ['C12','C2','C1']
    for i in range(len(df_features)):
        sub_df = df[df[df_features[i]].str.contains('^' + test_combination[i] + '$')]
        if sub_df.shape[0] > 0:
            # print("Pattern : " + df_features[i] + " , " + test_combination[i])
            return sub_df
    return df

In [169]:
# test_combination
test_comb = ['N AY JH IH R IY AH N', 'JH IH R IY AH N', 'N AY JH IH R IY', 'R IY AH N', 'JH IH R IY', 'N AY JH IH', 'AH N', 'R IY', 'JH IH', 'N AY']
sub_df = get_refined_df(test_comb, final_dataset)

Pattern : C1234 , N AY JH IH R IY AH N


In [49]:
sub_df

NameError: name 'sub_df' is not defined

## Now transform the testing set

In [226]:
train, test = train_test_split(final_dataset, test_size = 0.1)
features = list(final_dataset.columns)[11:]
syllables_comb = list(final_dataset.columns)[1:11]

test_y = list(test.Target)

In [58]:
def test_model(train, test, features):
    pred = []
    test_X_total = test[features]
    test_syllables_comb = test[syllables_comb]
    for i in range(test.shape[0]):
        curr_X = np.array(test_X_total.iloc[i]).reshape((1,-1))
        curr_syllables_comb = list(test_syllables_comb.iloc[i])
        
        sub_train = get_refined_df(curr_syllables_comb, train)
        train_X = sub_train[features]
        train_y = sub_train["Target"]
        
        if train_X.shape[0] < 5:
            neigh = KNeighborsClassifier(n_neighbors = train_X.shape[0], weights = 'distance')
            # neigh = KNeighborsClassifier(n_neighbors = 1, weights = 'distance')
            
        else:
            neigh = KNeighborsClassifier(n_neighbors = 5, weights = 'distance')
        neigh.fit(train_X, train_y)
        curr_pred = neigh.predict(curr_X)[0]
        pred.append(curr_pred)
    return pred

In [59]:
f1 = 0
for i in range(5):
    print("Round : " + str(i))
    train, test = train_test_split(final_dataset, test_size = 0.1)
    features = list(final_dataset.columns)[4:]
    syllables_comb = list(final_dataset.columns)[1:4]
    test_y = list(test.Target)
    pred = test_model(train, test, features)
    curr_f1 = f1_score(test_y, pred, average='macro')
    f1 += curr_f1
    print('F1 : ' + str(curr_f1))
print('Average F1 : ' + str(f1 * 1.0 / 5))

Round : 0
F1 : 0.805206220768
Round : 1
F1 : 0.795801249861
Round : 2
F1 : 0.800511559909
Round : 3
F1 : 0.818495411583
Round : 4
F1 : 0.794038820046
Average F1 : 0.802810652433
